In [23]:
from langchain.chains import LLMChain
from langchain_community.chat_models.azure_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv

In [24]:
load_dotenv()

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT_URL"],
    azure_deployment=os.environ["CHAT_MODEL"],
    openai_api_version=os.environ["API_VERSION"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
)


model.invoke("say hi")

AIMessage(content='Hi there! 👋 How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 9, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None}, id='run--0c82ba99-c828-4afc-8989-f2590c41f578-0')

In [25]:
from langchain_core.prompts import StringPromptTemplate, ChatPromptTemplate, PromptTemplate

prompt = ChatPromptTemplate.from_template('Say: {user_input}. My name is {name}.')

In [26]:
from langchain.chains import LLMChain
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_key='leonidas'
)

chain.invoke({"user_input": "hi", "name": "John"})

{'user_input': 'hi',
 'name': 'John',
 'leonidas': 'Hi, John! Nice to meet you! How can I assist you today? 😊'}

In [27]:
prompt.invoke({"name": "John", "user_input": "hi"})

ChatPromptValue(messages=[HumanMessage(content='Say: hi. My name is John.', additional_kwargs={}, response_metadata={})])

In [28]:
from langchain.chains import SequentialChain

In [29]:
prompt1 = ChatPromptTemplate.from_template(template='''Detect language of follwing message: {text}''')
chain1 = LLMChain(llm=model, prompt=prompt1, output_key='language')

prompt2 = ChatPromptTemplate.from_template(template='''Sumarize following message in 1 sentence: {text}''')
chain2 = LLMChain(llm=model, prompt=prompt2, output_key='summary')

prompt3 = ChatPromptTemplate.from_template(template='''Translate {summary} into {language}''')
chain3 = LLMChain(llm=model, prompt=prompt3, output_key='translated_summary')

In [30]:
chain = SequentialChain(
    chains=[chain1, chain2, chain3],
    input_variables=["text"],
    output_variables=['language', 'summary', 'translated_summary'],
    verbose=True
)

text = '''Do szkoły jedzie się całkiem długo, z powrotem nawet dłużej. W nocy, wracając skądś czasami, dużo mniej, choć więcej trzeba czekać na przystanku. No i jeszcze tyle samo jedzie się tam, skąd później wracać się będzie nocnymi. Jeszcze czasami idzie się pieszo załatwić sprawy, które można załatwić niedaleko domu. Te spacery należy również wliczyć w czas poświęcony na transport. W porównaniu całym dostępnym mi czasem transport pochłania, nie bagatelny, ale też nie zbyt duży fragment. A jednak namysł, który przeprowadza się w transporcie, stanowił nie proporcjonalnie do czasu dużą część myślenia, względem czasu. Nie należy się temu dziwić, biorąc pod uwagę, że czas snu zadaj kłam jakiejkolwiek próbie mierzenia namysłu za pomocą czasu i proporcji. Wciąż jednak można zwrócić uwagę na jakość myślenia, mierzoną na przykład żwawością, z którą pomysły przychodzą do głowy, albo inną podobną miarą. Jakiejkolwiek by jej nie wybrać, zdaje mi się, że namysł przeprowadzany w transporcie ma niebagatelnie dużą miarę.'''

chain.invoke({'text': text})



> Entering new SequentialChain chain...

> Finished chain.


{'text': 'Do szkoły jedzie się całkiem długo, z powrotem nawet dłużej. W nocy, wracając skądś czasami, dużo mniej, choć więcej trzeba czekać na przystanku. No i jeszcze tyle samo jedzie się tam, skąd później wracać się będzie nocnymi. Jeszcze czasami idzie się pieszo załatwić sprawy, które można załatwić niedaleko domu. Te spacery należy również wliczyć w czas poświęcony na transport. W porównaniu całym dostępnym mi czasem transport pochłania, nie bagatelny, ale też nie zbyt duży fragment. A jednak namysł, który przeprowadza się w transporcie, stanowił nie proporcjonalnie do czasu dużą część myślenia, względem czasu. Nie należy się temu dziwić, biorąc pod uwagę, że czas snu zadaj kłam jakiejkolwiek próbie mierzenia namysłu za pomocą czasu i proporcji. Wciąż jednak można zwrócić uwagę na jakość myślenia, mierzoną na przykład żwawością, z którą pomysły przychodzą do głowy, albo inną podobną miarą. Jakiejkolwiek by jej nie wybrać, zdaje mi się, że namysł przeprowadzany w transporcie ma ni

In [50]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain


template1 = ''' 
{history}\n
{translated_user_input}
'''
memory = ConversationBufferMemory()

chat_template = ChatPromptTemplate.from_template(''' 
You are helpful assistan, that chats in {language}.

Current conversation:
{history}
Human: {human_msg}
''')

chat_chain = ConversationChain(llm=model, 
                               memory=memory,
                               verbose=True,
                               output_key='eng_response',)
                               #template=chat_template)

print(chat_chain.get_input_schema)

<bound method Chain.get_input_schema of ConversationChain(memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[])), verbose=True, llm=AzureChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000013710063890>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000013710AF8ED0>, model_kwargs={}, openai_api_key='DGnhYMWJiZJOFGK08rilHZwGhfWNUoviEx4IShhlZUFD4Nc8Z4gYJQQJ99BEACfhMk5XJ3w3AAAAACOG1CPd', openai_proxy='', azure_endpoint='https://mikolaj-marsy-3981-resource.openai.azure.com/', deployment_name='gpt-4o', openai_api_version='2024-12-01-preview', openai_api_type='azure'), output_key='eng_response', output_parser=StrOutputParser(), llm_kwargs={})>


In [ ]:
memory.clear()

language_detect_chain = LLMChain(llm=model,
                                 prompt=prompt1,
                                 temperature=0.0,
                                 output_key='language',)

chat_chain

